In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
import tqdm
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import matplotlib as mpl
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Helvetica']})
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 10})
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}\usepackage{amsmath}\usepackage{upgreek}"

## prepare data:

In [ ]:
from mc2.utils.data_plotting import plot_single_sequence, plot_hysteresis
from mc2.data_management import FrequencySet, MaterialSet, DataSet

In [ ]:
dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "ten_mat_data.pickle")

## deleting N49 from dataset for now, since the data is incomplete
# 50 kHz and 80 kHz are missing
# 320 kHz has no data at 25 degrees

available_materials = deepcopy(dataset.material_names)
print(available_materials)
print(len(available_materials))

available_materials.remove("N49")
print(available_materials)
print(len(available_materials))


dataset = dataset.filter_materials(available_materials)
assert dataset.material_names == available_materials

all_relevant_data = dataset.at_material("3C90").at_frequency(50_000).filter_temperatures([25])
all_relevant_data

In [ ]:
tau = 1 / (16)

t0 = 600
t1 = 2900
t2 = 7000

B = all_relevant_data.B[0, t0:t2]
H = all_relevant_data.H[0, t0:t1]
T = all_relevant_data.T[0]
t = np.linspace(0, (B.shape[0] -1) * tau, B.shape[0])

## Plot:

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(3.5, 2.5), sharex=True)

colors = plt.rcParams["axes.prop_cycle"]()
c1 = next(colors)["color"]
c2 = next(colors)["color"]

axs[0].plot(t, B, color=c1)
axs[1].plot(t[:-(t2-t1)], H, color=c2)

axs[1].plot(t[-(t2-t1):], all_relevant_data.H[0, t1:t2], color=c2, linestyle="dashed")

axs[0].set_ylabel("$B \mathrm{\; in \; T}$")
axs[1].set_ylabel("$H \mathrm{\; in \; A/m}$")

for ax in axs:
    ax.grid(True, which="both", alpha=0.3)
    ax.tick_params(which='both', axis="y", direction='in')
    ax.tick_params(which='both', axis="x", direction='in') 
    ax.set_xlim([-15, 410])
    
axs[-1].set_xlabel("$t \mathrm{\; in \; \\upmu s}$")

for ax, perc in zip(axs, [0.06, 0.08]):
    ax.vlines(
        x=[0.0, (t1-t0) * tau, (t2-t0) * tau] ,
        ymin = ax.get_ylim()[0] - (perc * ax.get_ylim()[0]),
        ymax=ax.get_ylim()[1],
        color="k",
        linestyles="dashed",
        linewidth=1,
    )

axs[0].text(5, -0.1, '$t_0$', color='k', verticalalignment="top")
axs[0].text((t1-t0) * tau + 5, -0.1, '$t_1$', color='k', verticalalignment="top")
axs[0].text((t2-t0) * tau -20, -0.1, '$t_2$', color='k', verticalalignment="top")

axs[1].text(5, -1, '$t_0$', color='k', verticalalignment="top")
axs[1].text((t1-t0) * tau + 5, -1, '$t_1$', color='k', verticalalignment="top")
axs[1].text((t2-t0) * tau - 20, -1, '$t_2$', color='k', verticalalignment="top")

axs[1].text(((t2-t0) + (t1-t0)) * tau / 2 - 5, -70, '$\\textbf{?}$', fontsize=20, color=c2, verticalalignment="center")

fig.tight_layout(h_pad=0.2, w_pad=0)
fig.align_ylabels(axs)

fig.savefig("B_H_prediction_example.pdf", bbox_inches='tight')
fig.savefig("B_H_prediction_example.png", bbox_inches='tight')
plt.show()